In [1]:
from packages import Exam
from packages import Unit
import numpy as np
path = ["./files/result/csv/top_sample.csv",
        "./files/result/csv/back_sample.csv"]

variables = [
    {"name":"w0", "value":1e2, "unit":Unit.N_per_m(), "function": "w0", "first difine": 0},
    {"name":"EI", "value":1e5, "unit":Unit.Nm2(), "function": "EI", "first difine": 0},
    {"name":"L", "value":1e1, "unit":Unit.m(), "function": "L", "first difine": 1},
    {"name":"x", "value":np.array(range()), "unit":Unit.m(), "function": "L", "first difine": 1},
    {"name":"RA", "value":None, "unit":Unit.N(), "function": "w0 * L", "first difine": 1},
    {"name":"RB", "value":None, "unit":Unit.N(), "function": "w0 * L", "first difine": 1},
    {"name":"a", "value":None, "unit":Unit.N_per_m(), "function": "w0 * x / L", "first difine": 2},

]

exam = Exam.Exam(path)
exam.get_marks()
exam.get_student()
#q1
exam.parts.append(Exam.SingleAlphabetPart(exam.marks[0][4:9, 1:27],["A","Q","B","H","P"], [2] * 5))
#q2
exam.parts.append(Exam.SingleAlphabetPart(exam.marks[0][10:20, 1:27],["A","E","B","C","H","I","J","M","N","O"], [2] * 10))
#q3
exam.parts.append(Exam.DualNumberPart(
    marks = np.vstack([exam.marks[0][21:37, 3:27], exam.marks[0][2:6, 31:55]]),
    corrects = [
        [(1e3, Unit.N()),(-3e3, Unit.Nm())],
        [(3e2, Unit.N()),(-5e2, Unit.Nm())],
        [(4e1, Unit.N()),(-4e1, Unit.N())],
        [(1e3, Unit.N()),(1e3, Unit.N())],
        [(-6e1, Unit.N()),(5e2, Unit.N())]
    ],
    allocations = [6] * 5
    ))
exam.scoring()

print(exam.score)

scoring start of Q 1
scoring start of Q 2
scoring start of Q 3
60.0


In [2]:
eval("np.array([1,1])")

array([1, 1])

In [ ]:
Unit.Unit().value

In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 26),
    ( 2, 28),
    ( 2, 56),
    ( 3, 24),
    (40,  2),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/back_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions = marker_positions,
                  binary_threshold=240,
                  monitor= True)
sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()

In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 28),
    ( 2, 56),
    (40,  2),
    (40,  4),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/top_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions=marker_positions,
                  binary_threshold=240,
                  monitor= True)

sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()
